In [ ]:
import math
import sys
import os

import IPython
import matplotlib.pylab as plt
import numpy as np
from scipy import signal

In [ ]:
firmware_dir = "/home/duembgen/Documents/PhD/workspace/crazyflie-firmware/"
out_name = firmware_dir + "src/modules/src/sound_cf2.c"

n_buffer = 2048
fs = 32000

In [ ]:
from bin_selection import select_frequencies

min_freq = 100
max_freq = 4400
t = 1 # duration of each note in seconds


freqs = np.fft.rfftfreq(n_buffer, 1/fs)
bins = select_frequencies(n_buffer=n_buffer, fs=fs, min_freq=min_freq, max_freq=max_freq)
print(freqs[bins])

In [ ]:
print(f"add the following in {out_name} around line 200: \n")

# 1Q per beat (quarter note = beat)
# bps = r(bpm) / 60 (beat per second)
# => seconds per Q: t = 60/r
r = round(60 / t)

print(r"static Melody sweep = {.bpm = " + f"{int(r)}" + r", .delay = 1, .notes = {", end="")
duration = 0
for i, freq in enumerate(freqs[bins]):
    if i > 0:
        print(r"{" + f"{int(round(freq))}, Q" + r"}, ", end="")
        duration += t
    else:
        # add longer duration in beginning to allow for experiment
        # to start
        print(r"{" + f"{int(round(freq))}, W" + r"}, ", end="")
        duration += t * 4
print(r"STOP}};")

In [ ]:
print(f'duration: {len(bins) * t} seconds')
print(f'duration: {duration}')